# 6.1: 1次元入力2クラス分類

## 6.1.1 問題設定

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# データ生成
np.random.seed(seed=0)  # 乱数を固定
X_min, X_max = 0, 2.5   # Xの範囲
N = 30                  # データ数
col = ['cornflowerblue', 'gray']  # グラフの色
X = np.zeros(N)         # 入力データ
T = np.zeros(N, dtype=np.uint8)  # 目標データ
prm_s = [0.4, 0.8]      # 分布の開始位置
prm_w = [0.8, 1.6]      # 分布の幅
prm_pi = 0.5            # クラス1の比率

## 6.1.4 ロジスティック回帰モデル

担当　5.7-5.8

5.6-6.1.5まで

担当 6.1.7
範囲 6.1.6-6.2.2
8/1 6.3.1-6.3.2